# LambdaMART in Python

This is an implementation of LambdaMART in Python using sklearn and pandas. This is for educational purposes. 

But a secondary goal in getting this into Python is to more easily hack the algorithm to try new ideas. For example, this [blog article on two-sided marketplaces](https://opensourceconnections.com/blog/2017/07/04/optimizing-user-product-match-economies/), perhaps as more of an online algorithm (retiring old trees in the ensemble, adding new ones over time), perhaps with different model architectures in the ensemble (BERTy transformery things?) but all that preserve some of the nice things about LambdaMART (directly optimizing a list-wise metric)

This is adapted from [RankLib](https://github.com/o19s/RankyMcRankFace/blob/master/src/ciir/umass/edu/learning/tree/LambdaMART.java#L444) based on [this paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/MSR-TR-2010-82.pdf) from Microsoft Research.

## Table of Contents

1. [Setup - TheMovieDB corpus and log Elasticsearch features from TMDB](#Part-Zero---Setup---Get-TheMovieDB-Corpus-and-Log-Simple-Features) - plumbing to interact with Elasticsearch Learning to Rank to log a few basic features for our exploration
2. [Pairwise swapping](#Part-One---Collect-pair-wise-DCG-diffs) - here we demonstrate the core operation of LambdaMART - pairwise swapping of pairs and examining DCG (or another ranking metric) impact
3. [Scale to learn errors, not just swaps](#Part-Two---Compute-the-swaps-but-scaled-to-current-model's-error) - here we show how LambdaMART isn't just about learning the pairwise DCG difference of a swap, but the error currently in the model at predicting the DCG impact of that swap
4. [Weigh predictions](#Part-Three---Weigh-each-leaf's-predictions) - here we weigh the predictions of the next model in the ensemble based on how much DCG remains to be learned. 
5. [Putting it all together](#Part-Four---Putting-it-all-together,-from-the-top!) - the full algorithm in one place. You can also compare this notebook's output and learning to Ranklib.

---

6. [A Pandas version!](#5.-Pure-Pandas-Implementation?) -- walking through a faster version computing the per-tree training data using Pandas - a much for useful toy example.

## Known Issues

I'm still learning the nooks and crannies of the algorithm. So there are some known issues as this is actively being developed.

1. **Performance** - a single training round takes about 9 seconds. There's room for improvement in the hot part of the loop (dcg computation and swapping)

## Part Zero - Setup - Get TheMovieDB Corpus and Log Simple Features

In this step we download TheMovieDB Corpus and log some featurs (title and overview BM25). At the end we have a simple dataframe

In [11]:
from ltr.client import ElasticClient
client = ElasticClient()

### Download and index TMDB corpus and training set

Download [TheMovieDB](http://themoviedb.org) corpus and small toy training set with 40 queries labeled.

In [12]:
from ltr import download
corpus='http://es-learn-to-rank.labs.o19s.com/tmdb.json'
judgments='http://es-learn-to-rank.labs.o19s.com/title_judgments.txt'

download([corpus, judgments], dest='data/');
from ltr.index import rebuild
from ltr.helpers.movies import indexable_movies

movies=indexable_movies(movies='data/tmdb.json')
rebuild(client, index='tmdb', doc_src=movies)

data/tmdb.json already exists
data/title_judgments.txt already exists
Index tmdb already exists. Use `force = True` to delete and recreate


### Log two features - title & overview

Using the Elasticsearch Learning to Rank plugin, we:

1. Log two features: title and overview bm25
2. Create a pandas dataframe containing the labels and features

In [13]:
from ltr.log import FeatureLogger
from ltr.judgments import judgments_open
from itertools import groupby
from ltr.judgments import judgments_to_dataframe

client.reset_ltr(index='tmdb')

config = {"validation": {
              "index": "tmdb",
              "params": {
                  "keywords": "rambo"
              }
    
           },
           "featureset": {
            "features": [
                { #1
                    "name": "title_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"title": "{{keywords}}"}
                    }
                },
                { #2
                    "name": "overview_bm25",
                    "params": ["keywords"],
                    "template": {
                        "match": {"overview": "{{keywords}}"}
                    }
                }
            ]
    }}


client.create_featureset(index='tmdb', name='movies', ftr_config=config)

# Log features for each query
ftr_logger=FeatureLogger(client, index='tmdb', feature_set='movies')
with judgments_open('data/title_judgments.txt') as judgment_list:
    for qid, query_judgments in groupby(judgment_list, key=lambda j: j.qid):
        ftr_logger.log_for_qid(judgments=query_judgments, 
                               qid=qid,
                               keywords=judgment_list.keywords(qid))
        
# Convert to Pandas Dataframe
judgments = judgments_to_dataframe(ftr_logger.logged, unnest=False)

Removed Default LTR feature store [Status: 200]
Initialize Default LTR feature store [Status: 200]
Create movies feature set [Status: 201]
Recognizing 40 queries in: data/title_judgments.txt


### Examine judgments dataframe

In the dataframe we have a set of (query, document, grade) that label how relevant a document (movie) is for each query.

* qid - 'query id' - a unique identifier for this query
* docId - an identifier for the document (here movie) being labeled
* grade - how relevant a movie is on a 0-4 scale
* keywords - the query keywords that go along with the query id
* features - the two features we logged, 0th is title_bm25, 1st is overview_bm25

In [14]:
judgments

,uid,qid,keywords,docId,grade,features
0,1_7555,1,rambo,7555,4,"[11.657399, 10.083591]"
1,1_1370,1,rambo,1370,3,"[9.456276, 13.265001]"
2,1_1369,1,rambo,1369,3,"[6.036743, 11.113943]"
3,1_13258,1,rambo,13258,2,"[0.0, 6.869545]"
4,1_1368,1,rambo,1368,4,"[0.0, 11.113943]"
...,...,...,...,...,...,...
1385,40_37079,40,star wars,37079,0,"[0.0, 0.0]"
1386,40_126757,40,star wars,126757,0,"[0.0, 0.0]"
1387,40_39797,40,star wars,39797,0,"[0.0, 0.0]"
1388,40_18112,40,star wars,18112,0,"[0.0, 0.0]"


## Part One - Collect pair-wise DCG diffs

The first-pass iteration of LambdaMART, for each query, we examine the DCG\* impact of swapping each result with another result in the listing.

\* replace DCG with your metric of interest: MAP, Precision@N, etc

In [15]:
from math import log, exp
import numpy as np 

def rank_with_swap(ranked_list, rank1=0, rank2=0):
    """ Set the display rank of positions given the provided swap """
    ranked_list['display_rank'] = ranked_list.index.to_series()
    
    if rank1 != rank2:
        ranked_list.loc[rank1, 'display_rank'] = rank2
        ranked_list.loc[rank2, 'display_rank'] = rank1
    return ranked_list
    

def dcg(ranked_list, at=10):
    """Given a list, compute DCG -- 
       uses same variant as lambdamart 2**grade / log2(displayrank)
    """
    ranked_list['discount'] = 1 / np.log2(2 + ranked_list['display_rank'])
    ranked_list['gain'] = (2**ranked_list['grade'] - 1) * ranked_list['discount'] # TODO - precompute gain on swapping
    return sum(ranked_list['gain'].head(at))

def compute_swaps(query_judgments, axis, metric=dcg, at=10):
    """Compute the 'lambda' the DCG impact of every query result swapped with every-other query result"""
    
    # Sort to see ideal ordering
    # This isn't strictly nescesarry, but it's helpful to understand the algorithm
    query_judgments = query_judgments.sort_values('grade', kind='stable', ascending=False).reset_index()

    # Instead of explicitly 'swapping' we just swap the 'display_rank' - where 
    # in the final ranking this would be placed. We can easily use that to compute DCG
    query_judgments['display_rank'] = query_judgments.index.to_series()
    query_judgments['dcg'] = metric(query_judgments, at=at)
    best_dcg = query_judgments.loc[0, 'dcg']

    query_judgments['lambda'] = 0.0
    
    # TODO - redo inner body as 
    for better in range(0,len(query_judgments)):
        for worse in range(0,len(query_judgments)):
            if better > at and worse > at:
                break

            if query_judgments.loc[better, 'grade'] > query_judgments.loc[worse, 'grade']:
                query_judgments = rank_with_swap(query_judgments, better, worse)
                query_judgments['dcg'] = metric(query_judgments, at=at)

                dcg_after_swap = query_judgments.loc[0, 'dcg']
                delta = abs(best_dcg - dcg_after_swap)

                if delta > 0.0:

                    # Add delta to better's lambda (-delta to worse's lambda)
                    query_judgments.loc[better, 'lambda'] += delta
                    query_judgments.loc[worse, 'lambda'] -= delta

    # print(query_judgments[['keywords', 'docId', 'grade', 'lambda', 'features']])
    return query_judgments

# For each query, compute lambdas
# %prun -s cumulative lambdas_per_query = judgments.groupby('qid').apply(compute_swaps, axis=1)
# judgments
lambdas_per_query = judgments.groupby('qid').apply(compute_swaps, axis=1)
lambdas_per_query

index        uid  qid   keywords   docId  grade  \
qid                                                       
1   0       0     1_7555    1      rambo    7555      4   
    1       4     1_1368    1      rambo    1368      4   
    2       1     1_1370    1      rambo    1370      3   
    3       2     1_1369    1      rambo    1369      3   
    4       3    1_13258    1      rambo   13258      2   
...       ...        ...  ...        ...     ...    ...   
40  25   1385   40_37079   40  star wars   37079      0   
    26   1386  40_126757   40  star wars  126757      0   
    27   1387   40_39797   40  star wars   39797      0   
    28   1388   40_18112   40  star wars   18112      0   
    29   1389   40_43052   40  star wars   43052      0   

                      features  display_rank  discount       gain        dcg  \
qid                                                                            
1   0   [11.657399, 10.083591]             0  1.000000  15.000000  33.734341   
    1         [0.0, 11.113943]             1  0.630930   9.463946  33.734341   
    2    [9.456276, 13.265001]             2  0.500000   3.500000  33.734341   
    3    [6.036743, 11.113943]             3  0.430677   3.014736  33.734341   
    4          [0.0, 6.869545]             4  0.386853   1.160558  33.734341   
...                        ...           ...       ...        ...        ...   
40  25              [0.0, 0.0]            25  0.210310   0.000000  31.225149   
    26              [0.0, 0.0]            26  0.208015   0.000000  31.225149   
    27              [0.0, 0.0]            27  0.205847   0.000000  31.225149   
    28              [0.0, 0.0]            28  0.203795   0.000000  31.225149   
    29              [0.0, 0.0]             8  0.301030   0.000000  31.225149   

            lambda  
qid                 
1   0   427.587115  
    1   219.800566  
    2    62.204093  
    3    43.694734  
    4     5.542298  
...            ...  
40  25  -20.598524  
    26  -20.715586  
    27  -20.826142  
    28  -20.930783  
    29  -21.030027  

[1390 rows x 12 columns]

### Look at Precision instead of DCG

We can really use any ranking metric to achieve goals important to our product. This includes potentially ones we invent or come up with ourselves!

In [16]:
def precision(ranked_list, max_grade=4.0, at=10):
    """Given a list, compute simple precision. Really this is cumalitive gain."""
    above_n = ranked_list[ranked_list['display_rank'] < at]
    
    if (max_grade * at) == 0.0:
        print("0")
        return 0.0
    
    return float(sum(above_n['grade'])) / (max_grade * at)


lambdas_per_query_prec = judgments.groupby('qid').apply(compute_swaps, axis=1, metric=precision)
lambdas_per_query_prec.loc[5, :]

,index,uid,qid,keywords,docId,grade,features,display_rank,dcg,lambda
0,149,5_603,5,matrix,603,4,"[11.657399, 10.040129]",0,0.3,2.300
1,150,5_604,5,matrix,604,3,"[9.456276, 9.392262]",1,0.3,1.725
2,151,5_605,5,matrix,605,3,"[9.456276, 0.0]",2,0.3,1.725
3,152,5_55931,5,matrix,55931,2,"[0.0, 10.798681]",3,0.3,1.150
4,172,5_73262,5,matrix,73262,1,"[0.0, 0.0]",32,0.3,0.575
5,153,5_1857,5,matrix,1857,0,"[0.0, 9.65805]",5,0.3,0.000
6,154,5_10999,5,matrix,10999,0,"[0.0, 11.466951]",6,0.3,0.000
7,155,5_4247,5,matrix,4247,0,"[0.0, 8.114125]",7,0.3,0.000
8,156,5_21874,5,matrix,21874,0,"[0.0, 7.8627386]",8,0.3,0.000
9,157,5_181886,5,matrix,181886,0,"[0.0, 0.0]",9,0.3,0.000


### Fit a model on the lambdas

The core operation is fitting an operation on the lambdas (the accumulated pairwise differences)

In [21]:
train_set = lambdas_per_query[['lambda', 'features']]
train_set

lambda                features
qid                                       
1   0   427.587115  [11.657399, 10.083591]
    1   219.800566        [0.0, 11.113943]
    2    62.204093   [9.456276, 13.265001]
    3    43.694734   [6.036743, 11.113943]
    4     5.542298         [0.0, 6.869545]
...            ...                     ...
40  25  -20.598524              [0.0, 0.0]
    26  -20.715586              [0.0, 0.0]
    27  -20.826142              [0.0, 0.0]
    28  -20.930783              [0.0, 0.0]
    29  -21.030027              [0.0, 0.0]

[1390 rows x 2 columns]

In [22]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

tree = DecisionTreeRegressor()
tree.fit(train_set['features'].tolist(), train_set['lambda'])

DecisionTreeRegressor()

### DCG-based Lambda Predictions

We show predicting some known examples. In the first case, strong title and overview scores. In the second case, no title or overview scores

In [23]:
tree.predict([[11.1, 10.08]])

array([345.6500854])

In [24]:
tree.predict([[0.0, 0.0]])

array([-15.3987952])

It's more typical we would restrict the complexity of each tree in the ensemble. We can dump the tree see [understanding sklearn's tree structure](https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py)

In [25]:
tree = DecisionTreeRegressor(max_leaf_nodes=4)
tree.fit(train_set['features'].tolist(), train_set['lambda'])
plot_tree(tree)

[Text(248.0, 308.0, 'X[0] <= 10.666\nmse = 4161.543\nsamples = 1390\nvalue = -0.0'),
 Text(124.0, 184.79999999999998, 'X[0] <= 9.182\nmse = 1038.341\nsamples = 1329\nvalue = -9.745'),
 Text(62.0, 61.599999999999966, 'mse = 421.042\nsamples = 1301\nvalue = -11.724'),
 Text(186.0, 61.599999999999966, 'mse = 21086.54\nsamples = 28\nvalue = 82.191'),
 Text(372.0, 184.79999999999998, 'X[0] <= 18.186\nmse = 25061.546\nsamples = 61\nvalue = 212.311'),
 Text(310.0, 61.599999999999966, 'mse = 26150.423\nsamples = 51\nvalue = 188.147'),
 Text(434.0, 61.599999999999966, 'mse = 1343.167\nsamples = 10\nvalue = 335.547')]

## Part Two - Compute the swaps but _scaled_ to current model's error

LambdaMART is an _ensemble_ model. It's not just about the first model, but collecting a series of models where each model makes a gradual improvement on the current model. The technique used is known as [Gradient Boosting]()

To build a model that compensates for the current model's error, we scale the next set of dependent vars to predict based on the correctness of the existing model in ranking. In this way, we eliminate where the model currently does a good job (no need to learn these) and leave in places where the model isn't doing a good job (this is where. we want ot learn)

In [26]:
learning_rate = 0.1
judgments['last_prediction'] = tree.predict(judgments['features'].tolist()) * learning_rate

def compute_swaps_scaled(query_judgments, axis, metric=dcg, at=10):
    """Compute the 'lambda' the DCG impact of every query result swapped with every-other query result
    
    
    """
    
    # Important - stable sort. Otherwise DCG swaps get kind of wonky due to position discounts
    query_judgments = query_judgments.sort_values('last_prediction', ascending=False, kind='stable').reset_index()

    # Instead of explicitly 'swapping' we just swap the 'display_rank' - where 
    # in the final ranking this would be placed. We can easily use that to compute DCG
    query_judgments['display_rank'] = query_judgments.index.to_series()
    query_judgments['dcg'] = metric(query_judgments, at=at)
    best_dcg = query_judgments.loc[0, 'dcg']

    query_judgments['lambda'] = 0.0
    query_judgments['delta'] = 0.0
    
    for better in range(0,len(query_judgments)):
        for worse in range(better+1,len(query_judgments)):
            if better > at and worse > at:
                break
            if query_judgments.loc[better, 'grade'] > query_judgments.loc[worse, 'grade']:
                swap_judgments = rank_with_swap(query_judgments, better, worse)
                dcg_after_swap = metric(swap_judgments, at=at)

                delta = abs(best_dcg - dcg_after_swap)

                if delta > 0.0:
                    
                    # --------------
                    # NEW!
                    model_score_diff = query_judgments.loc[better, 'last_prediction'] - query_judgments.loc[worse, 'last_prediction']
                    rho = 1.0 / (1.0 + exp(model_score_diff))    
                    # --------------
                    # rho works as follows
                    # 
                    # model ranks                    rho
                    # better higher than worse       approaches 0      <-- model currently doing well!
                    # better same as worse.          0.5  
                    # worse higher than better       approaches 1      <-- model currently doing poorly!
                    # 
                    query_judgments.loc[better, 'delta'] += delta
                    
                    # Use rho to scale the lambdas
                    query_judgments.loc[better, 'lambda'] += delta * rho
        
                    query_judgments.loc[worse, 'lambda'] -= delta * rho
                    query_judgments.loc[worse, 'delta'] -= delta

    return query_judgments

judgments = judgments_to_dataframe(ftr_logger.logged, unnest=False)
judgments['last_prediction'] = 0.0
lambdas_per_query = judgments.groupby('qid').apply(compute_swaps_scaled, axis=1)
#
lambdas_per_query

index        uid  qid   keywords   docId  grade  \
qid                                                       
1   0       0     1_7555    1      rambo    7555      4   
    1       1     1_1370    1      rambo    1370      3   
    2       2     1_1369    1      rambo    1369      3   
    3       3    1_13258    1      rambo   13258      2   
    4       4     1_1368    1      rambo    1368      4   
...       ...        ...  ...        ...     ...    ...   
40  25   1385   40_37079   40  star wars   37079      0   
    26   1386  40_126757   40  star wars  126757      0   
    27   1387   40_39797   40  star wars   39797      0   
    28   1388   40_18112   40  star wars   18112      0   
    29   1389   40_43052   40  star wars   43052      0   

                      features  last_prediction  display_rank  discount  \
qid                                                                       
1   0   [11.657399, 10.083591]              0.0             0  1.000000   
    1    [9.456276, 13.265001]              0.0             1  0.630930   
    2    [6.036743, 11.113943]              0.0             2  0.500000   
    3          [0.0, 6.869545]              0.0             3  0.430677   
    4         [0.0, 11.113943]              0.0             4  0.386853   
...                        ...              ...           ...       ...   
40  25              [0.0, 0.0]              0.0            25  0.210310   
    26              [0.0, 0.0]              0.0            26  0.208015   
    27              [0.0, 0.0]              0.0            27  0.205847   
    28              [0.0, 0.0]              0.0            28  0.203795   
    29              [0.0, 0.0]              0.0             9  0.289065   

             gain        dcg      lambda       delta  
qid                                                   
1   0   15.000000  30.700871  213.776822  427.553645  
    1    4.416508  30.700871   48.987136   97.974272  
    2    3.500000  30.700871   31.835338   63.670676  
    3    1.292030  30.700871    6.723500   13.447000  
    4    5.802792  30.700871   41.948006   83.896012  
...           ...        ...         ...         ...  
40  25   0.000000  30.207651   -9.846078  -19.692155  
    26   0.000000  30.207651   -9.903461  -19.806921  
    27   0.000000  30.207651   -9.957655  -19.915309  
    28   0.000000  30.207651  -10.008949  -20.017899  
    29   0.000000  30.207651  -10.057598  -20.115197  

[1390 rows x 14 columns]

Zero in on 2 swapped by each result worse than it in query `ramba`

```
better_grade worse_grade, model_score_diffs, rho,                 dcg_delta
2 1                       0.758706128029972  0.31892724571177816  0.02502724555344038
2 1                       0.981162516523929  0.2726611758805124   0.04377062727053094
2 0                       1.142439045359345  0.24187283082372052  0.11698017724693699
2 0                       1.142439045359345  0.24187283082372052  0.14090604137532914
2 1                       1.142439045359345  0.24187283082372052  0.08043118677314176
2 1                       1.142439045359345  0.24187283082372052  0.17784892734690594
2 1                       1.142439045359345  0.24187283082372052  0.19254512210920538
2 1                       1.142439045359345  0.24187283082372052  0.20543079947538878
2 1                       1.142439045359345  0.24187283082372052  0.21685570619866112
2 0                       1.142439045359345  0.24187283082372052  0.22708156316293504
2 0                       1.142439045359345  0.24187283082372052  0.23630863576764227
2 0                       1.142439045359345  0.24187283082372052  0.24469312448475122
2 0                       1.142439045359345  0.24187283082372052  0.2523588995024131
2 0                       1.142439045359345  0.24187283082372052  0.25940563061320177
2 0                       1.142439045359345  0.24187283082372052  0.2659145510893115
...
2 0                       1.142439045359345  0.24187283082372052 0.34214193097965406
```

Summing all the model score diffs, we see those are rather high. This results in a high-ish rho between (for each value here 0.25-0.31). So each dcg_delta is added to the model.

What's the intuition here? The model hasn't entirely nailed this example, the model feels there's more 'dcg_delta' to learn to push it away from those less relevant results.

### Zooming out to more of `rambo`

We see a similar pattern in results with mediocre grades (2 and 3) where the resulting rho-scaled lambda's are higher than you might expect. The model's happy with the position of 0, but the ranking of other results could be separated more. The model diff should be higher when compared to the dcg diff to push the middling results away from the irrelevant result.

So the next tree learns these lambdas using the resulting features moreso than other results.

In [27]:
lambdas_per_query.loc[1, :][['keywords', 'display_rank',  'grade', 'last_prediction', 'delta', 'lambda', 'features']].sort_values('last_prediction', ascending=False)

,keywords,display_rank,grade,last_prediction,delta,lambda,features
0,rambo,0,4,0.0,427.553645,213.776822,"[11.657399, 10.083591]"
21,rambo,21,0,0.0,-19.868699,-9.934349,"[0.0, 0.0]"
23,rambo,23,0,0.0,-20.149295,-10.074647,"[0.0, 0.0]"
24,rambo,24,0,0.0,-20.276314,-10.138157,"[0.0, 0.0]"
25,rambo,25,0,0.0,-20.395685,-10.197842,"[0.0, 0.0]"
26,rambo,26,0,0.0,-20.508155,-10.254078,"[0.0, 0.0]"
27,rambo,27,1,0.0,-20.336529,-10.168264,"[0.0, 0.0]"
28,rambo,28,1,0.0,-20.432963,-10.216481,"[0.0, 0.0]"
29,rambo,29,1,0.0,-20.524423,-10.262211,"[0.0, 0.0]"
30,rambo,30,1,0.0,-20.611330,-10.305665,"[0.0, 0.0]"


In [28]:
from sklearn.tree import DecisionTreeRegressor

train_set = lambdas_per_query[['lambda', 'features']]
train_set

tree2 = DecisionTreeRegressor()
tree2.fit(train_set['features'].tolist(), train_set['lambda'])

DecisionTreeRegressor()

### More 'oomph' in second tree for the last tree's error cases

We see in the following lambdas our next tree learns more about the areas the last model seemed to need correction. 

The first example is well covered by the first tree.

In [29]:
tree2.predict([[11.6, 10.08]])

array([173.48027244])

The second example reflects some of the middling ranked results

In [30]:
tree2.predict([[0.0, 6.869545]])

array([6.72350002])

## Part Three - Weigh each leaf's predictions

Because we're dealing with trees, each leaf corresponds to a set of examples that have been grouped to this node. In addition to per-swap 'rho' we also care about a per-swap 'weight', referred to in gradient boosting as 'gamma'. 

Gamma means picking a weight for this sub-model that best predicts the final function.

First we group by the paths in the tree to uniquely identify each leaf

In [31]:
def compute_swaps_scaled_with_weights(query_judgments, axis, metric=dcg, at=10):
    """Compute the 'lambda' the DCG impact of every query result swapped with every-other query result
    
    
    """
    
    # Sort to see ideal ordering
    # This isn't strictly nescesarry, but it's helpful to understand the algorithm
    query_judgments = query_judgments.sort_values('last_prediction', ascending=False, kind='stable').reset_index()

    # Instead of explicitly 'swapping' we just swap the 'display_rank' - where 
    # in the final ranking this would be placed. We can easily use that to compute DCG
    query_judgments['display_rank'] = query_judgments.index.to_series()
    query_judgments['train_dcg'] = query_judgments['dcg'] = metric(query_judgments, at=at)
    train_dcg = query_judgments.loc[0, 'dcg']
 
    qid = query_judgments.loc[0, 'qid']
    keywords = query_judgments.loc[0, 'keywords']


    query_judgments['lambda'] = 0.0
    query_judgments['weight'] = 0.0

    for better in range(0,len(query_judgments)):
         for worse in range(0,len(query_judgments)):
            if better > at and worse > at:
                return query_judgments
                
            if query_judgments.loc[better, 'grade'] > query_judgments.loc[worse, 'grade']:
                query_judgments = rank_with_swap(query_judgments, better, worse)
                query_judgments['dcg'] = metric(query_judgments, at=at)

                dcg_after_swap = query_judgments.loc[0, 'dcg']
                delta = abs(train_dcg - dcg_after_swap)

                if delta != 0.0:
                    last_model_score_diff = query_judgments.loc[better, 'last_prediction'] - query_judgments.loc[worse, 'last_prediction']
                    rho = 1.0 / (1.0 + exp(last_model_score_diff)) 

                    assert(delta >= 0.0)
                    assert(rho >= 0.0)
                   
                    query_judgments.loc[better, 'lambda'] += delta * rho
                    query_judgments.loc[worse, 'lambda'] -= delta * rho
            
                    # --------------
                    # NEW!
                    #  last_model_score_diff        rho         weight
                    #      0.0                      0.5         0.25 (max possible value)
                    #      100.0                    0.0000      0.0  (max possible value)
                    # 
                    # If the current model has an ambiguous prediction, we include more of the delta in the weight
                    # If the current model has a strong prediction, weight approaches 0
                    query_judgments.loc[better, 'weight'] += rho * (1.0 - rho) * delta;
                    query_judgments.loc[worse, 'weight'] += rho * (1.0 - rho) * delta;
                    #
                    # These will be used to rescale each decision tree node's predictions
                    # If many results in a leaf node have last model score ~ ambiguous
                    #     the resulting model will have a high denominator ~ (1 / deltaDCG)
                    # If many results in a leaf node have last model score - not ambiguous, positive
                    #     the resulting model will have a low denominator
                    #
                    # Apparently we want to cancel out the deltas if last model was ambiguous?
                    # ---------------

                    

    return query_judgments

# Convert to Pandas Dataframe
judgments = judgments_to_dataframe(ftr_logger.logged, unnest=False)
judgments['last_prediction'] = 0
lambdas_per_query = judgments.groupby('qid').apply(compute_swaps_scaled_with_weights, axis=1)
lambdas_per_query = lambdas_per_query.drop('qid', axis=1).reset_index().drop(['level_1', 'index'], axis=1)

lambdas_per_query

,qid,uid,keywords,docId,grade,features,last_prediction,display_rank,discount,gain,train_dcg,dcg,lambda,weight
0,1,1_7555,rambo,7555,4,"[11.657399, 10.083591]",0,0,1.000000,15.000000,30.700871,30.552986,213.776822,106.888411
1,1,1_1370,rambo,1370,3,"[9.456276, 13.265001]",0,1,0.630930,4.416508,30.700871,30.552986,48.010828,26.458003
2,1,1_1369,rambo,1369,3,"[6.036743, 11.113943]",0,2,0.500000,3.500000,30.700871,30.552986,31.382749,18.143963
3,1,1_13258,rambo,13258,2,"[0.0, 6.869545]",0,3,0.430677,1.292030,30.700871,30.552986,6.460558,7.448315
4,1,1_1368,rambo,1368,4,"[0.0, 11.113943]",0,4,0.386853,5.802792,30.700871,30.552986,43.639845,21.819923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1385,40,40_37079,star wars,37079,0,"[0.0, 0.0]",0,25,0.210310,0.000000,30.207651,30.120435,-9.846078,4.923039
1386,40,40_126757,star wars,126757,0,"[0.0, 0.0]",0,26,0.208015,0.000000,30.207651,30.120435,-9.903461,4.951730
1387,40,40_39797,star wars,39797,0,"[0.0, 0.0]",0,27,0.205847,0.000000,30.207651,30.120435,-9.957655,4.978827
1388,40,40_18112,star wars,18112,0,"[0.0, 0.0]",0,28,0.203795,0.000000,30.207651,30.120435,-10.008949,5.004475


In [32]:
from sklearn.tree import DecisionTreeRegressor

train_set = lambdas_per_query[['lambda', 'features']]
train_set

tree3 = DecisionTreeRegressor(max_leaf_nodes=4)
tree3.fit(train_set['features'].tolist(), train_set['lambda'])
        

DecisionTreeRegressor(max_leaf_nodes=4)

### Label each row with its unique prediction (ie tree path)

In [33]:
def tree_paths(tree, X):
    paths_as_array = tree.decision_path(X).toarray()
    paths = ["".join(item) for item in paths_as_array.astype(str)]
    return paths

lambdas_per_query['path'] = tree_paths(tree3, train_set['features'].tolist())

### Override outputs using our own weighted average

The typical decision tree uses either the [median or mean of the target values](https://scikit-learn.org/stable/modules/tree.html#regression-criteria) classified to a given leaf node as the prediction. However, in the case of lambdaMART, we want to use a weighted average that accounts for how much of the DCG error out there has been accounted for. Thus the psuedoresponses are summed and divided by the remaining error DCG.

rho=0, then an example is weighed by `1/0.25*deltaNDCG` as there's a lot of outstanding DCG error left.

In [34]:
lambdas_per_query.groupby('path')['weight'].sum()

path
1010001    1673.720249
1010010    1467.050422
1100100     538.598514
1101000    4655.114588
Name: weight, dtype: float64

In [35]:
lambdas_per_query.groupby('path')['lambda'].sum()

path
1010001    3334.073882
1010010    2787.629391
1100100     893.120752
1101000   -7014.824025
Name: lambda, dtype: float64

In [ ]:
round_predictions = lambdas_per_query.groupby('path')['lambda'].sum() / lambdas_per_query.groupby('path')['weight'].sum()
round_predictions.to_dict()

In [ ]:
plot_tree(tree3)

### Wrap the original tree, and provide new value

Instead of directly using the provided decision tree, we want to use our prediction for each leaf. This `OverridenDecisionTree` takes the original tree, looks up the prediction path in the original tree, but uses our values for the predicted variable instead of what's here.

In [46]:
class OverridenRegressionTree:
    def __init__(self, predictions, tree):
        self.predictions = predictions
        self.tree = tree
        
    def predict(self, X, use_original=False):
        if use_original:
            return self.predict(X)
        path = self.tree.decision_path(X).toarray().astype(str)
        path = "".join(path[0])
        
        paths_as_array = self.tree.decision_path(X).toarray()
        paths = ["".join(item) for item in paths_as_array.astype(str)]
        
        predictions = self.predictions[paths]
        
        # Any NaN predictions is a red flag, debug
        if np.any(predictions.isnull()):
            print(predictions[predictions.isnull()])
            print(pd.DataFrame(X)[predictions.isnull().reset_index(drop=True)])
            raise AssertionError("No prediction should be NaN")
        return np.array(self.predictions[paths].tolist())
        
override_tree = OverridenRegressionTree(predictions = round_predictions, tree=tree3)
override_tree.predict([[0.0, 6.869545], [10.0, 10.0]])

NameError: name 'round_predictions' is not defined

## Part Four - Putting it all together, from the top!

Now we can put together the full lambdamart algorithm that 

1. Uses pair-wise swaps on our our metric (ie DCG) to generate decision tree predictors (the 'lambdas')
2. Focuses in on predicting where current model makes the wrong call when ranking by DCG
3. Predicts using a weighted average, weighed by 1 / (remaining DCG)

TODOs / known issues
* While DCG converges, it does sometimes wander a tad, so there might be more room for improvement
* Speeding up the inner loop of the `compute_swaps_scaled_with_weights` that must run for ever query's swaps

In [ ]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd

def predict(ensemble, X, learning_rate=0.1):
    prediction = 0
    for tree in ensemble:
        prediction += tree.predict(X) * learning_rate
    return prediction.rename('prediction')


def tree_paths(tree, X):
    paths_as_array = tree.decision_path(X).toarray()
    paths = ["".join(item) for item in paths_as_array.astype(str)]
    return paths

ensemble=[]
def lambda_mart(judgments, rounds=20, learning_rate=0.1, max_leaf_nodes=8, metric=dcg):

    print(judgments.columns)
    # Convert to Pandas Dataframe
    lambdas_per_query = judgments.copy()


    lambdas_per_query['last_prediction'] = 0.0

    for i in range(0, rounds):
        print(f"round {i}")

        # ------------------
        #1. Build pair-wise predictors for this round
        lambdas_per_query = lambdas_per_query.groupby('qid').apply(compute_swaps_scaled_with_weights, 
                                                                   axis=1, metric=dcg)

        # ------------------
        #2. Train a regression tree on this round's lambdas
        features = lambdas_per_query['features'].tolist()
        tree = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
        tree.fit(features, lambdas_per_query['lambda'])    

        # ------------------
        #3. Reweight based on LambdaMART's weighted average
        # Add each tree's paths
        lambdas_per_query['path'] = tree_paths(tree, features)
        predictions = lambdas_per_query.groupby('path')['lambda'].sum() / lambdas_per_query.groupby('path')['weight'].sum()
        predictions = predictions.fillna(0.0) # for divide by 0

        # -------------------
        #4. Add to ensemble, recreate last prediction
        new_tree = OverridenRegressionTree(predictions=predictions, tree=tree)
        ensemble.append(new_tree)
        next_predictions = new_tree.predict(features)
        lambdas_per_query['last_prediction'] += (next_predictions * learning_rate) 
        
        print("Train DCGs")
        print("mean   ", lambdas_per_query['train_dcg'].mean())
        print("median ", lambdas_per_query['train_dcg'].median())
        print("----------")

        
        # Reset the dataframe for further processing

        lambdas_per_query = lambdas_per_query.drop('qid', axis=1).reset_index().drop(['level_1', 'index'], axis=1)

judgments = judgments_to_dataframe(ftr_logger.logged, unnest=False)
lambdas_per_query = lambda_mart(judgments=judgments, rounds=50, max_leaf_nodes=10, learning_rate=0.1, metric=dcg)

In [ ]:
lambdas_per_query.iloc[282]

In [ ]:
ensemble[0].predictions

In [ ]:
plot_tree(ensemble[0].tree)

## Compare to ranklib output

In [ ]:
from ltr.ranklib import train
trainLog  = train(client,
                  training_set=ftr_logger.logged,
                  index='tmdb',
                  trees=10,
                  featureSet='movies',
                  modelName='title')

print("Every N rounds of ranklib")
trainLog.trainingLogs[0].rounds

## Examine queries we learned

Try out some queries, look at the final model prediction `last_prediction` compare to the correct ordering `grade`.

In [ ]:
lambdas_per_query[lambdas_per_query['qid'] == 2]

# 5. Pure Pandas Implementation?

Can we make it faster by vectorizing with pandas?

Turns out Yes!

In [36]:
lambdas_per_query = judgments.copy()


lambdas_per_query['last_prediction'] = 0.0
lambdas_per_query.sort_values(['qid', 'last_prediction'], ascending=[True, False], kind='stable')

,uid,qid,keywords,docId,grade,features,last_prediction
0,1_7555,1,rambo,7555,4,"[11.657399, 10.083591]",0.0
1,1_1370,1,rambo,1370,3,"[9.456276, 13.265001]",0.0
2,1_1369,1,rambo,1369,3,"[6.036743, 11.113943]",0.0
3,1_13258,1,rambo,13258,2,"[0.0, 6.869545]",0.0
4,1_1368,1,rambo,1368,4,"[0.0, 11.113943]",0.0
...,...,...,...,...,...,...,...
1385,40_37079,40,star wars,37079,0,"[0.0, 0.0]",0.0
1386,40_126757,40,star wars,126757,0,"[0.0, 0.0]",0.0
1387,40_39797,40,star wars,39797,0,"[0.0, 0.0]",0.0
1388,40_18112,40,star wars,18112,0,"[0.0, 0.0]",0.0


In [38]:
lambdas_per_query.sort_values(['qid', 'last_prediction'], ascending=[True, False], kind='stable')
lambdas_per_query['display_rank'] = lambdas_per_query.groupby('qid').cumcount()

#TBD - How do generalize this?
lambdas_per_query['discount'] = 1 / np.log2(2 + lambdas_per_query['display_rank'])
lambdas_per_query['gain'] = (2**lambdas_per_query['grade'] - 1) # * lambdas_per_query['discount']

lambdas_per_query[['qid', 'display_rank', 'discount', 'grade', 'gain']]

,qid,display_rank,discount,grade,gain
0,1,0,1.000000,4,15
1,1,1,0.630930,3,7
2,1,2,0.500000,3,7
3,1,3,0.430677,2,3
4,1,4,0.386853,4,15
...,...,...,...,...,...
1385,40,25,0.210310,0,0
1386,40,26,0.208015,0,0
1387,40,27,0.205847,0,0
1388,40,28,0.203795,0,0


## Pairwise deltas

Delta captures pair-wise difference of the ranking metric (ie DCG)


In [39]:
# each group paired with each other group
swaps = lambdas_per_query.merge(lambdas_per_query, on='qid', how='outer')
# changes[j][i] = changes[i][j] = (discount(i) - discount(j)) * (gain(rel[i]) - gain(rel[j]));
swaps['delta'] = np.abs((swaps['discount_x'] - swaps['discount_y']) * (swaps['gain_x'] - swaps['gain_y']))
swaps[['qid', 'display_rank_x', 'display_rank_y', 'delta']]

,qid,display_rank_x,display_rank_y,delta
0,1,0,0,0.000000
1,1,0,1,2.952562
2,1,0,2,4.000000
3,1,0,3,6.831881
4,1,0,4,0.000000
...,...,...,...,...
49019,40,29,25,0.000000
49020,40,29,26,0.000000
49021,40,29,27,0.000000
49022,40,29,28,0.000000


## Pairwise rhos

Rho captures pair-wise difference of the current model's prediction.


In [40]:
swaps['rho'] = 1 / (1 + np.exp(swaps['last_prediction_x'] - swaps['last_prediction_y']))
swaps[['qid', 'display_rank_x', 'display_rank_y', 'delta', 'last_prediction_x', 'last_prediction_y', 'rho']]

,qid,display_rank_x,display_rank_y,delta,last_prediction_x,last_prediction_y,rho
0,1,0,0,0.000000,0.0,0.0,0.5
1,1,0,1,2.952562,0.0,0.0,0.5
2,1,0,2,4.000000,0.0,0.0,0.5
3,1,0,3,6.831881,0.0,0.0,0.5
4,1,0,4,0.000000,0.0,0.0,0.5
...,...,...,...,...,...,...,...
49019,40,29,25,0.000000,0.0,0.0,0.5
49020,40,29,26,0.000000,0.0,0.0,0.5
49021,40,29,27,0.000000,0.0,0.0,0.5
49022,40,29,28,0.000000,0.0,0.0,0.5


## Compute lambdas

For every row where grade_x > grade_y,  compute `delta*rho`

In [41]:
swaps['lambda'] = 0
slice_x_better =swaps[swaps['grade_x'] > swaps['grade_y']]
swaps.loc[swaps['grade_x'] > swaps['grade_y'], 'lambda'] = slice_x_better['delta'] * slice_x_better['rho']
swaps[['qid', 'display_rank_x', 'display_rank_y', 'delta', 'last_prediction_x', 'last_prediction_y', 'rho', 'lambda']]

,qid,display_rank_x,display_rank_y,delta,last_prediction_x,last_prediction_y,rho,lambda
0,1,0,0,0.000000,0.0,0.0,0.5,0.000000
1,1,0,1,2.952562,0.0,0.0,0.5,1.476281
2,1,0,2,4.000000,0.0,0.0,0.5,2.000000
3,1,0,3,6.831881,0.0,0.0,0.5,3.415941
4,1,0,4,0.000000,0.0,0.0,0.5,0.000000
...,...,...,...,...,...,...,...,...
49019,40,29,25,0.000000,0.0,0.0,0.5,0.000000
49020,40,29,26,0.000000,0.0,0.0,0.5,0.000000
49021,40,29,27,0.000000,0.0,0.0,0.5,0.000000
49022,40,29,28,0.000000,0.0,0.0,0.5,0.000000


## Get per-key lambdas

We merge back together the xs minuse the ys

In [51]:
# Better minus worse
lambdas_x = swaps.groupby(['qid', 'display_rank_x'])['lambda'].sum().rename('lambda')
lambdas_y = swaps.groupby(['qid', 'display_rank_y'])['lambda'].sum().rename('lambda')
lambdas = lambdas_x - lambdas_y
lambdas
lambdas_per_query = lambdas_per_query.merge(lambdas, left_on=['qid', 'display_rank'], right_on=['qid', 'display_rank_x'], how='left')
lambdas_per_query[['qid', 'docId', 'grade', 'features', 'lambda']]

,qid,docId,grade,features,lambda
0,1,7555,4,"[11.657399, 10.083591]",211.781688
1,1,1370,3,"[9.456276, 13.265001]",46.938369
2,1,1369,3,"[6.036743, 11.113943]",30.637615
3,1,13258,2,"[0.0, 6.869545]",5.888732
4,1,1368,4,"[0.0, 11.113943]",43.177578
...,...,...,...,...,...
1385,40,37079,0,"[0.0, 0.0]",-9.853045
1386,40,126757,0,"[0.0, 0.0]",-9.911575
1387,40,39797,0,"[0.0, 0.0]",-9.966853
1388,40,18112,0,"[0.0, 0.0]",-10.019174


In [43]:
lambdas_per_query.merge(lambdas, left_on=['qid', 'display_rank'], right_on=['qid', 'display_rank_x'], how='left')

,uid,qid,keywords,docId,grade,features,last_prediction,display_rank,discount,gain,lambda
0,1_7555,1,rambo,7555,4,"[11.657399, 10.083591]",0.0,0,1.000000,15,211.781688
1,1_1370,1,rambo,1370,3,"[9.456276, 13.265001]",0.0,1,0.630930,7,46.938369
2,1_1369,1,rambo,1369,3,"[6.036743, 11.113943]",0.0,2,0.500000,7,30.637615
3,1_13258,1,rambo,13258,2,"[0.0, 6.869545]",0.0,3,0.430677,3,5.888732
4,1_1368,1,rambo,1368,4,"[0.0, 11.113943]",0.0,4,0.386853,15,43.177578
...,...,...,...,...,...,...,...,...,...,...,...
1385,40_37079,40,star wars,37079,0,"[0.0, 0.0]",0.0,25,0.210310,0,-9.853045
1386,40_126757,40,star wars,126757,0,"[0.0, 0.0]",0.0,26,0.208015,0,-9.911575
1387,40_39797,40,star wars,39797,0,"[0.0, 0.0]",0.0,27,0.205847,0,-9.966853
1388,40_18112,40,star wars,18112,0,"[0.0, 0.0]",0.0,28,0.203795,0,-10.019174


In [53]:
#2. Train a regression tree on this round's lambdas
features = lambdas_per_query['features'].tolist()
tree = DecisionTreeRegressor(max_leaf_nodes=10)
tree.fit(features, lambdas_per_query['lambda'])    

tree

DecisionTreeRegressor(max_leaf_nodes=10)

In [ ]:
ensemble.append(tree)


In [44]:
def compute_lambdas(lambdas_per_query):
    lambdas_per_query = lambdas_per_query.sort_values(['qid', 'last_prediction'], ascending=[True, False], kind='stable')
    lambdas_per_query['display_rank'] = lambdas_per_query.groupby('qid').cumcount()

    #TBD - How do generalize this to any metric?
    lambdas_per_query['discount'] = 1 / np.log2(2 + lambdas_per_query['display_rank'])
    lambdas_per_query['gain'] = (2**lambdas_per_query['grade'] - 1)

    # swaps dataframe holds each pair-wise swap computed (shrink columns for memory?)   
    # Optimization of swaps = lambdas_per_query.merge(lambdas_per_query, on='qid', how='outer')
    # to limit to just needed columns
    to_swap = lambdas_per_query[['qid', 'display_rank', 'grade', 'last_prediction', 'discount', 'gain']]
    #to_swap = lambdas_per_query
    swaps = to_swap.merge(to_swap, on='qid', how='outer')

    # delta - delta in DCG due to swap
    swaps['delta'] = np.abs((swaps['discount_x'] - swaps['discount_y']) * (swaps['gain_x'] - swaps['gain_y']))
    
    # rho - based on current model prediction delta
    swaps['rho'] = 1 / (1 + np.exp(swaps['last_prediction_x'] - swaps['last_prediction_y']))
    
    # If you want to be pure gradient boosting, weight reweights each models prediction
    # I haven't found this to matter in practice
    swaps['weight'] = swaps['rho'] * (1.0 - swaps['rho']) * swaps['delta']

    # Compute lambdas (the next model in ensemble's predictors) when grade_x > grade_y
    swaps['lambda'] = 0
    slice_x_better =swaps[swaps['grade_x'] > swaps['grade_y']]
    swaps.loc[swaps['grade_x'] > swaps['grade_y'], 'lambda'] = slice_x_better['delta'] * slice_x_better['rho']
    
    # accumulate lambdas and add back to model
    lambdas_x = swaps.groupby(['qid', 'display_rank_x'])['lambda'].sum().rename('lambda')
    lambdas_y = swaps.groupby(['qid', 'display_rank_y'])['lambda'].sum().rename('lambda')

    weights_x = swaps.groupby(['qid', 'display_rank_x'])['weight'].sum().rename('weight')
    weights_y = swaps.groupby(['qid', 'display_rank_y'])['weight'].sum().rename('weight')
    
    weights = weights_x + weights_y
    lambdas = lambdas_x - lambdas_y

    lambdas_per_query = lambdas_per_query.merge(lambdas, 
                                                left_on=['qid', 'display_rank'], 
                                                right_on=['qid', 'display_rank_x'], 
                                                how='left')
    lambdas_per_query = lambdas_per_query.merge(weights, 
                                                left_on=['qid', 'display_rank'], 
                                                right_on=['qid', 'display_rank_x'], 
                                                how='left')

    return lambdas_per_query

In [47]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd


ensemble=[]
def lambda_mart_pure(judgments, rounds=20, learning_rate=0.1, max_leaf_nodes=8, metric=dcg):

    print(judgments.columns)
    # Convert to Pandas Dataframe
    lambdas_per_query = judgments.copy()


    lambdas_per_query['last_prediction'] = 0.0

    for i in range(0, rounds):
        print(f"round {i}")

        # ------------------
        #1. Build pair-wise predictors for this round
        lambdas_per_query = compute_lambdas(lambdas_per_query)

        # ------------------
        #2. Train a regression tree on this round's lambdas
        features = lambdas_per_query['features'].tolist()
        tree = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes)
        tree.fit(features, lambdas_per_query['lambda'])    

        # ------------------
        #3. Reweight based on LambdaMART's weighted average
        # Add each tree's paths
        lambdas_per_query['path'] = tree_paths(tree, features)
        predictions = lambdas_per_query.groupby('path')['lambda'].sum() / lambdas_per_query.groupby('path')['weight'].sum()
        predictions = predictions.fillna(0.0) # for divide by 0

        # -------------------
        #4. Add to ensemble, recreate last prediction
        new_tree = OverridenRegressionTree(predictions=predictions, tree=tree)
        ensemble.append(new_tree)
        next_predictions = new_tree.predict(features)
        lambdas_per_query['last_prediction'] += (next_predictions * learning_rate) 
        
        print(lambdas_per_query.loc[0, ['grade', 'last_prediction']])
        
        print("Train DCGs")
        lambdas_per_query['discounted_gain'] = lambdas_per_query['gain'] * lambdas_per_query['discount'] 
        dcg = lambdas_per_query[lambdas_per_query['display_rank'] < 10].groupby('qid')['discounted_gain'].sum().mean()
        print("mean   ", dcg)
        print("----------")
        
        lambdas_per_query = lambdas_per_query.drop(['lambda', 'weight'], axis=1)
    return lambdas_per_query


judgments = judgments_to_dataframe(ftr_logger.logged, unnest=False)
lambdas_per_query = lambda_mart_pure(judgments=judgments, rounds=50, max_leaf_nodes=10, learning_rate=0.01, metric=dcg)

Index(['uid', 'qid', 'keywords', 'docId', 'grade', 'features'], dtype='object')
round 0
grade                 4
last_prediction    0.01
Name: 0, dtype: object
Train DCGs
mean    20.193092700772873
----------
round 1
grade                     4
last_prediction    0.019915
Name: 0, dtype: object
Train DCGs
mean    20.709013438589473
----------
round 2
grade                     4
last_prediction    0.029745
Name: 0, dtype: object
Train DCGs
mean    20.709013438589473
----------
round 3
grade                     4
last_prediction    0.039492
Name: 0, dtype: object
Train DCGs
mean    20.709013438589473
----------
round 4
grade                     4
last_prediction    0.049159
Name: 0, dtype: object
Train DCGs
mean    20.709013438589473
----------
round 5
grade                     4
last_prediction    0.058748
Name: 0, dtype: object
Train DCGs
mean    20.709013438589473
----------
round 6
grade                    4
last_prediction    0.06826
Name: 0, dtype: object
Train DCGs
mean    20.70901

In [ ]:
judgments = judgments_to_dataframe(ftr_logger.logged, unnest=False)
%prun -s cumtime lambdas_per_query = lambda_mart_pure(judgments=judgments, rounds=50, max_leaf_nodes=10, learning_rate=0.01, metric=dcg)

In [18]:
from ltr.ranklib import train
%prun -s cumtime trainLog  = train(client, training_set=ftr_logger.logged, index='tmdb', trees=10, featureSet='movies', modelName='title')

print("Every N rounds of ranklib")
trainLog.trainingLogs[0].rounds

GET http://es-learn-to-rank.labs.o19s.com/RankyMcRankFace.jar
Running java -jar /var/folders/bz/schh49y93yg6323ynrz7j8j00000gn/T/RankyMcRankFace.jar -ranker 6 -shrinkage 0.1 -metric2t DCG@10 -tree 10 -bag 1 -leaf 10 -frate 1.0 -srate 1.0 -train /var/folders/bz/schh49y93yg6323ynrz7j8j00000gn/T/training.txt -save data/title_model.txt 
Delete model title: 404
Created Model title [Status: 201]
Model saved
 Every N rounds of ranklib


[20.692,
 20.669,
 20.669,
 20.669,
 20.669,
 20.73,
 20.7506,
 20.6507,
 20.7237,
 20.7237]

In [ ]:
class OverridenRegressionTree:
    def __init__(self, predictions, tree):
        self.predictions = predictions
        self.tree = tree
        
    def predict(self, X, use_original=False):
        if use_original:
            return self.predict(X)
        path = self.tree.decision_path(X).toarray().astype(str)
        path = "".join(path[0])
        
        paths_as_array = self.tree.decision_path(X).toarray()
        paths = ["".join(item) for item in paths_as_array.astype(str)]
        
        predictions = self.predictions[paths]
        
        # Any NaN predictions is a red flag, debug
        if np.any(predictions.isnull()):
            print(predictions[predictions.isnull()])
            print(pd.DataFrame(X)[predictions.isnull().reset_index(drop=True)])
            raise AssertionError("No prediction should be NaN")
        return np.array(self.predictions[paths].tolist())